In [94]:
import pandas as pd
from datetime import datetime, timedelta

# Set up location and time point

In [103]:
# some point in North Pacific Ocean
target_longitude = -170.2
target_latitude = 40.4
# how large should the region around the target location be for averaging values
region_padding_degrees = 0.0
# how many days back do we look for averaging values
days_back = 10

# current time
time_point = datetime.now()
# another point in time
#time_point = datetime(2023, 12, 25)

# Retrieve data from Copernicus API

In [106]:
import copernicusmarine

def format_data_points(current_datetime, days_back = 30):
    current_date_str = current_datetime.strftime("%Y-%m-%d")
    earlier_datetime = current_datetime - timedelta(days=days_back)
    earlier_date_str = earlier_datetime.strftime("%Y-%m-%d")
    return earlier_date_str, current_date_str

def copernicus_salinity_temp(target_longitude, region_padding_degrees, time_point):
    start_time, end_time = format_data_points(time_point)
    
    salinity_ds = copernicusmarine.open_dataset(
        dataset_id = "cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m",
        minimum_longitude = target_longitude-region_padding_degrees,
        maximum_longitude = target_longitude+region_padding_degrees,
        minimum_latitude = target_latitude-region_padding_degrees,
        maximum_latitude = target_latitude+region_padding_degrees,
        start_datetime = start_time,
        end_datetime = end_time,
        variables = ["sea_water_salinity"],
        # USER DATA ARE KEPT FOR HISTORY REASONS, CAN BE REPLACED WITH SOMETHING MEANINGFUL
        username='test',
        password='test'
    )
    salinity_ds = salinity_ds.to_dataframe().reset_index()
    
    temp_ds = copernicusmarine.open_dataset(
        dataset_id = "cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m",
        minimum_longitude = target_longitude-region_padding_degrees,
        maximum_longitude = target_longitude+region_padding_degrees,
        minimum_latitude = target_latitude-region_padding_degrees,
        maximum_latitude = target_latitude+region_padding_degrees,
        start_datetime = start_time,
        end_datetime = end_time,
        variables = ["sea_water_potential_temperature"],
        # USER DATA ARE KEPT FOR HISTORY REASONS, CAN BE REPLACED WITH SOMETHING MEANINGFUL
        username='test',
        password='test'
    )
    temp_ds = temp_ds.to_dataframe().reset_index()
    
    measurement_df = temp_ds.merge(salinity_ds, on = ['depth', 'latitude', 'longitude', 'time'])
    measurement_df = measurement_df.drop(columns = ['time']).groupby(['depth', 'latitude', 'longitude']).mean().reset_index()
    
    return measurement_df.dropna()

In [107]:
measurement_df = copernicus_salinity_temp(target_longitude, region_padding_degrees, time_point)
measurement_df

INFO - 2024-05-15T11:51:51Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-05-15T11:51:51Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-05-15T11:51:51Z - The dataset cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m, version '202211', will be retired on the 2024-07-26T00:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-05-15T11:51:52Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-05-15T11:51:55Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-05-15T11:51:55Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-05-15T11:51:55Z - The dataset cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m, version '202211', will be retired on the 2024-07-26T00:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-05-15T11:51:56Z - Service was not sp

,depth,latitude,longitude,thetao,so
0,0.494025,40.416672,-170.166672,11.932512,33.864967
1,1.541375,40.416672,-170.166672,11.920527,33.865219
2,2.645669,40.416672,-170.166672,11.907635,33.865643
3,3.819495,40.416672,-170.166672,11.893132,33.866055
4,5.078224,40.416672,-170.166672,11.876390,33.866123
5,6.440614,40.416672,-170.166672,11.859618,33.866047
6,7.929560,40.416672,-170.166672,11.841192,33.865944
7,9.572997,40.416672,-170.166672,11.826330,33.865894
8,11.405000,40.416672,-170.166672,11.813334,33.865894
9,13.467140,40.416672,-170.166672,11.800666,33.865891


# Compute sound speed

In [108]:
# Compute the speed of sound using Mackenzie’s formula
def compute_sound_speed(T, S, D):
    T_squared = T**2
    T_cubed = T**3
    D_squared = D**2
    D_cubed = D**3
    C = (1448.96 + 
         4.591*T - 
         5.304e-2*T_squared + 
         2.374e-4*T_cubed + 
         1.340*(S-35) + 
         1.630e-2*D + 
         1.675e-7*D_squared - 
         1.025e-2*T*(S-35) - 
         7.139e-13*T*D_cubed)
    return C

In [109]:
measurement_df['SOUND_SPEED'] = measurement_df.apply(lambda row: compute_sound_speed(row['thetao'], row['so'], row['depth']), axis=1)
measurement_df

,depth,latitude,longitude,thetao,so,SOUND_SPEED
0,0.494025,40.416672,-170.166672,11.932512,33.864967,1495.219349
1,1.541375,40.416672,-170.166672,11.920527,33.865219,1495.195511
2,2.645669,40.416672,-170.166672,11.907635,33.865643,1495.169682
3,3.819495,40.416672,-170.166672,11.893132,33.866055,1495.139414
4,5.078224,40.416672,-170.166672,11.876390,33.866123,1495.102380
5,6.440614,40.416672,-170.166672,11.859618,33.866047,1495.066737
6,7.929560,40.416672,-170.166672,11.841192,33.865944,1495.027397
7,9.572997,40.416672,-170.166672,11.826330,33.865894,1495.002899
8,11.405000,40.416672,-170.166672,11.813334,33.865894,1494.987955
9,13.467140,40.416672,-170.166672,11.800666,33.865891,1494.977872
